MODULE DOWNLOADS (IMPORTANT TO HAVE PYTORCH THAT IS CUDA COMPATIBLE)

In [2]:
%pip install pydicom
%pip install torchmetrics
%pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0 --extra-index-url https://download.pytorch.org/whl/cu113

Keyring is skipped due to an exception: 'keyring.backends'
Note: you may need to restart the kernel to use updated packages.
Keyring is skipped due to an exception: 'keyring.backends'
Note: you may need to restart the kernel to use updated packages.
Keyring is skipped due to an exception: 'keyring.backends'
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
Note: you may need to restart the kernel to use updated packages.


MORE GENERIC PREPROCESSING 

In [21]:
s3 = boto.client('s3')
sagemaker_session = sagemaker.Session()
bucket = 'mammographydata'
key_prefix = 'DataSet/processed/'
download_area = 'all_data' 
paginator = s3.get_paginator('list_objects_v2')
pages = paginator.paginate(Bucket= bucket, Prefix= key_prefix)
i = 0
j = 0
labels = pickle.load(open('label_dict.pt', 'rb'))
undownloaded_list = []
for label in labels.keys():
    if label not in os.listdir('all_data'):
        i += 1
    files = os.listdir(f'all_data/{label}')
    if  'LCC.pt' not in files or 'LMLO.pt' not in files or 'RCC_flipped.pt' not in files or 'RMLO_flipped.pt' not in files:
        print(label)
        j += 1
        undownloaded_list.append(label)
print(f'NUMBER OF UNDOWNLOADED DATA: {i}')
print(f'NUMBER OF ISSUES: {j}')

NUMBER OF UNDOWNLOADED DATA: 0
NUMBER OF ISSUES: 0


In [ ]:
!wget https://mammographydata.s3.amazonaws.com/totaldata.zip

In [ ]:
!unzip -q totaldata.zip

In [ ]:
df = pd.read_csv(f'parent/breast-level_annotations.csv')
df = df[(df['height'] == 3518) & (df['width'] == 2800)]
df = df[~df['study_id'].isin(['dbca9d28baa3207b3187c4d07dc81a80'])]

In [ ]:
def find_zeros(data):
    x , y = data.shape
    zero_t = torch.tensor(0)
    if x != 3500:
        diff = x - 3500
        i = 0
        j = 0
        k = 0
        while k != diff:
            #print(f'in loop {k} time on same data')
            if (torch.count_nonzero(data[:i,:]) == zero_t):
                i += 1
            if (torch.count_nonzero(data[x-j:x,:] == zero_t)):
                j += 1
            k += 1
            
        if i + j != diff:
            #print('There was an issue')
            i = 0
            j = diff
            
            
    return data[i:x-j,:]

In [ ]:
s3_client = boto.client('s3')
for patient in undownloaded_list:
    #print(patient)
    curr_df = df[df['study_id'] == patient]
    lmlo_image_id = list(curr_df[((curr_df['laterality'] == 'L') & (
        curr_df['view_position'] == 'MLO'))]['image_id'])[0]
    lcc_image_id = list(curr_df[((curr_df['laterality'] == 'L') & (
        curr_df['view_position'] == 'CC'))]['image_id'])[0]
    rmlo_image_id = list(curr_df[((curr_df['laterality'] == 'R') & (
        curr_df['view_position'] == 'MLO'))]['image_id'])[0]
    rcc_image_id = list(curr_df[((curr_df['laterality'] == 'R') & (
        curr_df['view_position'] == 'CC'))]['image_id'])[0]

    lmlo_dicom = dicom.dcmread(f'parent/images/{patient}/{lmlo_image_id}.dicom')
    lcc_dicom = dicom.dcmread(f'parent/images/{patient}/{lcc_image_id}.dicom')
    rmlo_dicom = dicom.dcmread(f'parent/images/{patient}/{rmlo_image_id}.dicom')
    rcc_dicom = dicom.dcmread(f'parent/images/{patient}/{rcc_image_id}.dicom')
    

    lmlo_torch = find_zeros(torch.from_numpy(np.array(lmlo_dicom.pixel_array, dtype= np.float32)))
    lcc_torch = find_zeros(torch.from_numpy(np.array(lcc_dicom.pixel_array, dtype = np.float32)))
    rmlo_torch = find_zeros(torch.flip(torch.from_numpy(np.array(rmlo_dicom.pixel_array, dtype = np.float32)),(1,)))
    rcc_torch = find_zeros(torch.flip(torch.from_numpy(np.array(rcc_dicom.pixel_array, dtype= np.float32)),(1,)))
    
    pickle.dump(lmlo_torch, open(f'upload/{patient}_LMLO.pt', "wb"))
    pickle.dump(lcc_torch, open(f'upload/{patient}_LCC.pt', "wb"))
    pickle.dump(rmlo_torch, open(f'upload/{patient}_RMLO_flipped.pt', "wb"))
    pickle.dump(rcc_torch, open(f'upload/{patient}_RCC_flipped.pt', "wb"))
    
    s3_client.upload_file(f'upload/{patient}_LMLO.pt', 'mammographydata', f'DataSet/processed/{patient}/LMLO.pt')
    s3_client.upload_file(f'upload/{patient}_LCC.pt', 'mammographydata', f'DataSet/processed/{patient}/LCC.pt')
    s3_client.upload_file(f'upload/{patient}_RMLO_flipped.pt', 'mammographydata', f'DataSet/processed/{patient}/RMLO_flipped.pt')
    s3_client.upload_file(f'upload/{patient}_RCC_flipped.pt', 'mammographydata', f'DataSet/processed/{patient}/RCC_flipped.pt')

GETTING DATA FROM BUCKET INTO CORRECT PROPER LOCAL DIRECTORY

In [ ]:
#Get Data In Local Directory
s3 = boto.client('s3')
sagemaker_session = sagemaker.Session()
bucket = 'mammographydata'
key_prefix = 'DataSet/processed/'
download_area = 'all_data' 
paginator = s3.get_paginator('list_objects_v2')
pages = paginator.paginate(Bucket= bucket, Prefix= key_prefix)
file_num = 0
page_num = 0
for page in pages:
    response = page['Contents']
    response = response[1:]
    for file in response:
        key = file['Key'].split('/')
        patient_ID = key[2]
        view = key[3]
        download_path = f'{download_area}/{patient_ID}'
        specific_prefix = f'{key_prefix}{patient_ID}/{view}'
        sagemaker_session.download_data(download_path, bucket, specific_prefix, extra_args=None)
        file_num += 1
        if file_num % 400 == 0:
            print(f'On file num: {file_num}')
    print(f'On Page: {page_num}')
    page_num += 1

BEGIN TRAINING

In [1]:
import boto3 as boto
import pickle
import torch
import sagemaker
import pandas as pd
import os
import numpy as np
import pydicom as dicom

In [ ]:
from mammography_project.src.models import data
from mammography_project.src.models import MyClasses
from mammography_project.src.models import PaperModel
from mammography_project.src.models import runners

In [ ]:
PaperModel.main(batch_size = 4)

LEN TEST GEN BEFORE: 762
LEN TRAIN GEN BEFORE: 3048
LEN TEST GEN: 760
LEN TRAIN GEN: 3048
Trying Batched
Starting Training
	[GPU cuda] Epoch 1
Have run 20 batches
Have run 40 batches
Have run 60 batches
Have run 80 batches
Have run 100 batches
Have run 120 batches
Have run 140 batches
Have run 160 batches
Have run 180 batches
Have run 200 batches
Have run 220 batches
Have run 240 batches
Have run 260 batches
Have run 280 batches
Have run 300 batches
Have run 320 batches
Have run 340 batches
Have run 360 batches
Have run 380 batches
Have run 400 batches
Have run 420 batches
Have run 440 batches
Have run 460 batches
Have run 480 batches
Have run 500 batches
Have run 520 batches
Have run 540 batches
Have run 560 batches
Have run 580 batches
Have run 600 batches
Have run 620 batches
Have run 640 batches
Have run 660 batches
Have run 680 batches
Have run 700 batches
Have run 720 batches
Have run 740 batches
Have run 760 batches
	Train Metrics (Training Data):
		Overall Loss: 0.9560247063636